In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
from datetime import date

In [8]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.3/917.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 37.1 MB/s eta 0:0

In [9]:
from darts import TimeSeries
from darts.models import TSMixerModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse


In [10]:
full_df = pd.read_csv('/content/drive/MyDrive/Omdena_Dengue_fever_Sri_Lanka/Sri Lanka Dengue models/Copy of Sri_lanka_dengue_cases_weather_weekly_2007_2024_ updated.csv')

In [11]:
full_df.head()

,District,Number_of_Cases,Week_Start_Date,Month,Year,Week,Week_End_Date,Avg Max Temp (°C),Avg Min Temp (°C),Avg Apparent Max Temp (°C),Avg Apparent Min Temp (°C),Total Precipitation (mm),Total Rain (mm),Avg Wind Speed (km/h),Max Wind Gusts (km/h),Weather Code,Avg Daylight Duration (hours),Avg Sunrise Time,Avg Sunset Time
0,Ampara,0,2007-01-01,1,2007,1,2007-01-08,26.9375,21.8625,28.8500,24.1000,8.0,8.0,18.2000,44.3,3,11.714583,376,1079
1,Ampara,0,2007-01-08,1,2007,2,2007-01-15,26.3500,23.3250,30.2625,26.7000,86.1,86.1,17.0250,43.6,53,11.733333,378,1082
2,Ampara,0,2007-01-15,1,2007,3,2007-01-22,27.2125,23.1625,29.5750,25.9625,3.1,3.1,20.1875,48.2,51,11.754167,380,1085
3,Ampara,0,2007-01-22,1,2007,4,2007-01-29,26.7250,23.0375,29.9500,26.1125,53.6,53.6,17.2000,42.5,61,11.783333,381,1088
4,Ampara,0,2007-01-29,1,2007,5,2007-02-05,27.3625,22.9625,30.7500,26.1250,8.2,8.2,17.2625,40.0,51,11.812500,382,1090


In [12]:
districts = full_df['District'].unique()
districts

array(['Ampara', 'Anuradhapura', 'Badulla', 'Batticaloa', 'Colombo',
       'Galle', 'Gampaha', 'Hambantota', 'Jaffna', 'Kalutara', 'Kandy',
       'Kegalle', 'Kilinochchi', 'Kurunegala', 'Mannar', 'Matale',
       'Matara', 'Monaragala', 'Mullaitivu', 'NuwaraEliya', 'Polonnaruwa',
       'Puttalam', 'Ratnapura', 'Trincomalee', 'Vavuniya'], dtype=object)

In [13]:
cols = ['District','Number_of_Cases', 'Week_Start_Date', 'Month', 'Avg Max Temp (°C)','Avg Min Temp (°C)', 'Total Precipitation (mm)',
        'Avg Wind Speed (km/h)','Max Wind Gusts (km/h)', 'Avg Daylight Duration (hours)']

In [14]:
df = full_df[cols]
df.head()

,District,Number_of_Cases,Week_Start_Date,Month,Avg Max Temp (°C),Avg Min Temp (°C),Total Precipitation (mm),Avg Wind Speed (km/h),Max Wind Gusts (km/h),Avg Daylight Duration (hours)
0,Ampara,0,2007-01-01,1,26.9375,21.8625,8.0,18.2000,44.3,11.714583
1,Ampara,0,2007-01-08,1,26.3500,23.3250,86.1,17.0250,43.6,11.733333
2,Ampara,0,2007-01-15,1,27.2125,23.1625,3.1,20.1875,48.2,11.754167
3,Ampara,0,2007-01-22,1,26.7250,23.0375,53.6,17.2000,42.5,11.783333
4,Ampara,0,2007-01-29,1,27.3625,22.9625,8.2,17.2625,40.0,11.812500


In [15]:
df['Week_Start_Date'] = pd.to_datetime(df['Week_Start_Date'])

## Univariate Forecast

In [14]:
ampara_df = df[df['District']=='Ampara'][['Week_Start_Date','Number_of_Cases']]
ampara_df.head()

,Week_Start_Date,Number_of_Cases
0,2007-01-01,0
1,2007-01-08,0
2,2007-01-15,0
3,2007-01-22,0
4,2007-01-29,0


In [34]:
model = TSMixerModel(
    input_chunk_length=24,           # length of input sequence
    output_chunk_length=12,
    batch_size=32,                   # batch size during training
    norm_type="LayerNorm",
    activation='SELU',
    n_epochs=100,                    # number of epochs for training
    random_state=42,
)

In [35]:
def forecast_univariate(district):
    district_df = df[df['District']==district][['Week_Start_Date','Number_of_Cases']]
    district_df.reset_index(drop=True, inplace=True)
    district_df = district_df.sort_values(by='Week_Start_Date')

    series = TimeSeries.from_dataframe(district_df, time_col='Week_Start_Date', value_cols=['Number_of_Cases'])
    train, test = series.split_after(pd.Timestamp('2024-04-22'))

    #scaler = Scaler()
    #train_scaled = scaler.fit_transform(train)
    #test_scaled = scaler.transform(test)

    model.fit(train)

    forecast = model.predict(n=12)

    err_rmse = rmse(test, forecast)

    #err_mape = mape(test, forecast)



    print(f"{district} RMSE: {err_rmse}")
    #print(f"{district} MAPE: {err_mape}")
    return err_rmse, #err_mape

In [16]:
# scaled data
ampara_rmse, ampara_mape = forecast_univariate('Ampara')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | criterion           | MSELoss          | 0      | train
1 | train_criterion     | MSELoss          | 0      | train
2 | val_criterion       | MSELoss          | 0      | train
3 | train_metrics       | MetricCollection | 0      | train
4 | val_metrics         | MetricCollection | 0      | train
5 | fc_hist             | Linear           | 300    | train
6 | feature_mixing_hist | _FeatureMixing   | 6.0 K  | train
7 | conditional_mixer   | ModuleList       | 23.1 K | train
8 | fc_out              | Linear           | 65     | train
----------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Ampara RMSE: 0.01523516359058474
Ampara MAPE: 97.94461442903226


Input without scaling

In [36]:
ALL_RMSE = []
#ALL_MAPE = []
for district in districts:
  rmse_error = forecast_univariate(district)
  ALL_RMSE.append(rmse_error)
  #ALL_MAPE.append(mape_error)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | criterion           | MSELoss          | 0      | train
1 | train_criterion     | MSELoss          | 0      | train
2 | val_criterion       | MSELoss          | 0      | train
3 | train_metrics       | MetricCollection | 0      | train
4 | val_metrics         | MetricCollection | 0      | train
5 | fc_hist             | Linear           | 300    | train
6 | feature_mixing_hist | _FeatureMixing   | 6.0 K  | train
7 | conditional_mixer   | ModuleList       | 23.1 K | train
8 | fc_out              | Linear           | 65     | train
----------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | criterion           | MSELoss          | 0      | train
1 | train_criterion     | MSELoss          | 0      | train
2 | val_criterion       | MSELoss          | 0      | train
3 | train_metrics       | MetricCollection | 0      | train
4 | val_metrics         | MetricCollection | 0      | train
5 | fc_hist             | Linear           | 300    | train
6 | feature_mixing_hist | _FeatureMixing   | 6.0 K  | train
7 | conditional_mixer   | ModuleList       | 23.1 K | train
8 | fc_out              | Linear           | 65     | train
----------------------------------------------

...

Trincomalee RMSE: 5.123838386117603


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Vavuniya RMSE: 14.64615979289895


In [3]:
ALL_RMSE = [
    5.556920157504543,
    4.909260972520963,
    6.837321019018064,
    9.63131964514407,
    118.0280116129935,
    28.70801012689876,
    70.7570805584996,
    13.675434297499399,
    11.217089800795042,
    20.612362111503085,
    17.24680002924321,
    15.364046438092283,
    19.650651306745416,
    20.587169931756772,
    26.322957899917395,
    17.476046333932278,
    18.901710910537734,
    15.933036461726104,
    13.750820860409211,
    8.338930475537216,
    10.705909701108833,
    6.5550944069655595,
    45.07449080022878,
    5.123838386117603,
    14.64615979289895
]

In [4]:
len(ALL_RMSE)

25

In [7]:
np.array(ALL_RMSE).mean()

21.824418961503774

## Multivariate Forecast

In [16]:
from darts.models import TSMixerModel

In [17]:
model = TSMixerModel(
    input_chunk_length=24,
    output_chunk_length=12,
    batch_size=32,                   # batch size during training
    norm_type="LayerNorm",
    activation='SELU',
    n_epochs=100,                    # number of epochs for training
    random_state=42
)

In [18]:
def forecast_multivariate(district):
    # create distric dataframe
    district_df = df[df['District'] == district]
    district_df.reset_index(drop=True, inplace=True)
    district_df = district_df.sort_values(by='Week_Start_Date')
    district_df.drop(['District'], axis=1, inplace=True)

    multi_series = TimeSeries.from_dataframe(district_df, time_col='Week_Start_Date', value_cols=['Number_of_Cases','Month','Avg Max Temp (°C)','Avg Min Temp (°C)','Total Precipitation (mm)','Avg Wind Speed (km/h)',
        'Max Wind Gusts (km/h)','Avg Daylight Duration (hours)'])

    train, test = multi_series.split_after(pd.Timestamp('2024-04-22'))

    target = train['Number_of_Cases']

    past_cov = train['Month']
    past_cov = past_cov.stack(train['Avg Max Temp (°C)'])
    past_cov = past_cov.stack(train['Avg Min Temp (°C)'])
    past_cov = past_cov.stack(train['Total Precipitation (mm)'])
    past_cov = past_cov.stack(train['Avg Wind Speed (km/h)'])
    past_cov = past_cov.stack(train['Max Wind Gusts (km/h)'])
    past_cov = past_cov.stack(train['Avg Daylight Duration (hours)'])

    model.fit(target, past_covariates=past_cov, verbose=True)
    pred = model.predict(12)

    err_rmse = rmse(test['Number_of_Cases'], pred)

    print(f"{district} RMSE: {err_rmse}")
    return err_rmse

In [19]:
ALL_MULTI_RMSE = []
for district in districts:
  rmse_error = forecast_multivariate(district)
  ALL_MULTI_RMSE.append(rmse_error)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | criterion           | MSELoss          | 0      | train
1 | train_criterion     | MSELoss          | 0      | train
2 | val_criterion       | MSELoss          | 0      | train
3 | train_metrics       | MetricCollection | 0      | train
4 | val_metrics         | MetricCollection | 0      | train
5 | fc_hist             | Linear           | 300    | train
6 | feature_mixing_hist | _FeatureMixing   | 6.8 K  | train
7 | conditional_mixer   | ModuleList       | 23.1 K | train
8 | fc_out              | Linear           | 65     | train
----------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | criterion           | MSELoss          | 0      | train
1 | train_criterion     | MSELoss          | 0      | train
2 | val_criterion       | MSELoss          | 0      | train
3 | train_metrics       | MetricCollection | 0      | train
4 | val_metrics         | MetricCollection | 0      | train
5 | fc_hist             | Linear           | 300    | train
6 | feature_mixing_hist | _FeatureMixing   | 6.8 K  | train
7 | conditional_mixer   | ModuleList       | 23.1 K | train
8 | fc_out              | Linear           | 65     | train
----------------------------------------------

...

Trincomalee RMSE: 7.356051476724751


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Vavuniya RMSE: 4.402545437871275


In [20]:
np.array(ALL_MULTI_RMSE).mean()

18.863252587420735